<a href="https://colab.research.google.com/github/MariaZharova/Skillfactory_projects/blob/main/SASRec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ВНИМАНИЕ! Для более быстрой работы ноутбука рекомендуем подключить встроенную в колаб видеокарту. Это можно сделать, выбрав в верхнем меню "изменить" -> "настройки блокнота" -> "аппаратный ускоритель - GPU" -> "сохранить".

Сначала установим необходимые библиотеки, которых нет по умолчанию в колабе

In [1]:
! pip install scrapbook category_encoders recommenders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.0/339.0 KB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.1/353.1 KB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 KB 865.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 630.1/630.1 KB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 KB 21.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 57.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━

In [2]:
import re
import sys
import os
import scrapbook as sb
from tempfile import TemporaryDirectory
import numpy as np
import pandas as pd 

from collections import defaultdict
import category_encoders as ce
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.datasets.amazon_reviews import get_review_data
from recommenders.datasets.split_utils import filter_k_core

# Transformer Based Models
from recommenders.models.sasrec.model import SASREC
from recommenders.models.sasrec.sampler import WarpSampler
from recommenders.models.sasrec.util import SASRecDataSet

In [3]:
# считываем данные
clicks = pd.read_csv('/content/drive/MyDrive/SF/clicks.csv')

In [4]:
# кодируем пользователей и объекты, начиная с единицы
offer_encoder = {off: ind for ind, off in enumerate(clicks['offer_id'].unique())}
clicks['offer_id_enc'] = clicks['offer_id'].map(offer_encoder) + 1
uid_encoder = {uid: ind for ind, uid in enumerate(clicks['user_id'].unique())}
clicks['user_id_enc'] = clicks['user_id'].map(uid_encoder) + 1

# сортируем клики пользователей по времени
clicks['timestamp'] = pd.to_datetime(clicks['timestamp'])
clicks.sort_values(by=['user_id_enc', 'timestamp'], inplace=True)
clicks.head(10)

,timestamp,user_id,offer_id,event_name,offer_id_enc,user_id_enc
1651310,2022-07-11 16:33:51.215,82662998,272458509,OpenOfferScreen,313343,1
1919327,2022-07-11 16:35:28.448,82662998,269087803,OpenOfferScreen,21269,1
1927186,2022-07-11 16:37:37.318,82662998,275502080,OpenOfferScreen,89189,1
3573155,2022-07-11 16:55:04.947,82662998,275170855,OpenOfferScreen,2125,1
3453458,2022-07-13 20:05:53.175,82662998,275743706,OpenOfferScreen,183187,1
1901171,2022-07-21 18:57:17.236,82662998,275343838,OpenOfferScreen,807,1
4058801,2022-07-21 18:57:36.242,82662998,275049379,OpenOfferScreen,129275,1
4058929,2022-07-21 19:01:16.998,82662998,276145959,OpenOfferScreen,68822,1
4177867,2022-07-21 19:05:51.014,82662998,275226154,OpenOfferScreen,100936,1
1475947,2022-07-21 19:07:23.352,82662998,273552270,OpenOfferScreen,77557,1


In [5]:
# сохраняем .txt файл, который будем подавать на вход модели
clicks[['offer_id_enc',	'user_id_enc']].to_csv('input_for_model.txt', sep="\t", header=False, index=False)

In [6]:
# составляем датасет специального формата из сохранённого текстового файла
data = SASRecDataSet(filename='input_for_model.txt', col_sep='\t')
# аналог train_test_split
data.split()

In [7]:
# задаём гиперпараметры модели - можете "поиграться" с ними
# и посмотреть на изменение качества модели :)
num_epochs = 5        # количество эпох обучения
batch_size = 512       # размер батча
RANDOM_SEED = 42       # random state 

lr = 0.001             # скорость обучения
maxlen = 50            # длина одного эмбеддинга
num_blocks = 2         # количество блоков-трансформеров
hidden_units = 100     # количеств рассматриваемых неявных признаков (аналог из SVD)
num_heads = 1          # количество слоёв самовнимания
dropout_rate = 0.1     # один из способов регуляризации в нейросетях - "отключение" доли нейронов
l2_emb = 0.0           # коэффициент L2-регуляризации
num_neg_test = 100     # количество "негативных" кликов (т.е. те объявления, на которые пользователь не нажимал),
                       # нужно для составления непосредственно рекомендаций


In [8]:
# функция библиотеки для возможности распараллелить обучение, изменить размер батча (порции данных для обучения)
# и др. технических параметров обучения
sampler = WarpSampler(data.user_train, data.usernum, data.itemnum, \
                      batch_size=batch_size, maxlen=maxlen, n_workers=3)

In [9]:
# инициализация модели
model = SASREC(item_num=data.itemnum,
               seq_max_len=maxlen,
               num_blocks=num_blocks,
               embedding_dim=hidden_units,
               attention_dim=hidden_units,
               attention_num_heads=num_heads,
               dropout_rate=dropout_rate,
               conv_dims = [100, 100],
               l2_reg=l2_emb,
               num_neg_test=num_neg_test)

In [10]:
# обучение + сразу оценивание (уже встроено в train)
with Timer() as train_time:
    t_test = model.train(data, sampler, num_epochs=num_epochs, batch_size=batch_size, lr=lr, val_epoch=3)

Evaluating...



epoch: 5, time: 672.961949915, valid (NDCG@10: 0.9008405621067385, HR@10: 0.973404255319149)
epoch: 5, time: 672.961949915,  test (NDCG@10: 0.8848116675867963, HR@10: 0.9692126909518214)


Evaluating...



epoch: 10, time: 1144.079213049, valid (NDCG@10: 0.9079466622558294, HR@10: 0.9753986332574032)
epoch: 10, time: 1144.079213049,  test (NDCG@10: 0.8905602258679031, HR@10: 0.9630727148392979)



epoch: 10, test (NDCG@10: 0.8998505229826427, HR@10: 0.9735582402552998)


Если уже сейчас хотите подробнее разобраться с принципом работы модели, можете рассмотреть этот пример от самих разработчиков https://github.com/microsoft/recommenders/blob/main/examples/00_quick_start/sasrec_amazon.ipynb